In [1]:
# Requires ipympl
%matplotlib widget

import numpy as np
from scipy import stats
import matplotlib.pyplot as plt
import ipywidgets as widgets

plt.ioff()

g = 2.002319 # Gyromagnetic ratio constant of a free electron.
h = 6.62607015*10**-34 # Planck's constant 6.62607015 × 10-34 m2 kg / s
nu = 9.4*10**9 # EPR instrument frequency 9.4 GHz
muB = 9.2740100783*10**-24 # Bohr magneton 9.2740100783(28)×10−24 J / T
B = h*nu/(g*muB) # Magnetic field strength at which transition occurs.

splits = [[B, 1]]
def reset(b):
    global splits
    
    # First element is middle point of peak, second element is the peaks proportion of spectrums tot. probability
    splits = [[B, 1]]

# splitPeaks performs a splitting of the input based on hyperfine constant, number of equivalent nuclei and their spin
def splitPeaks(hfconst, nuclei, spin):
#     print(spin)
    spinx2 = int(2*spin)
    pattern = np.zeros((nuclei+1, 2*spinx2+1+(nuclei-1)*spinx2))
    pattern[0][spinx2] = 1
    for row in range(1, len(pattern)):
        for column in range(spinx2, len(pattern[row])):
            pattern[row][column] = np.sum(pattern[row-1][column-spinx2:column+1])
        
#     print(pattern)
#     print(len(pattern[-1]))
    newSplits = []
    nonZeroesInLastRow = [x for x in pattern[-1] if x != 0]
    leftPointOffset = (len(nonZeroesInLastRow)-1)/2*hfconst
    for e in splits:
        for point in range(len(nonZeroesInLastRow)):
            newSplits.append([e[0]-leftPointOffset+hfconst*point, nonZeroesInLastRow[point]/nuclei])
    return newSplits

def setPattern(b):
    reset(None)
    
    global splits
#     for e in range(len(HFDict)):
    for e in range(ActiveSplits):
        splits = splitPeaks(HFDict[e].value, NucleiDict[e].value, SpinDict[e].value)

HFDict, NucleiDict, SpinDict = {}, {}, {}
def plot(resolution, xMin, xMax, firstDeriv, NumberOfSplits):
    x = np.linspace(xMin, xMax, resolution) # All points making up the x-axis.
    y = np.zeros(resolution) # Generate y-axis where peaks are added to later on.
    
    # Used for keeping track how many splits the user has selected.
    global ActiveSplits
    ActiveSplits = NumberOfSplits
    
    global HFDict, NucleiDict, SpinDict
    for e in range(NumberOfSplits):
        
        # If statements used to avoid overwriting split settings when user interacts with widgets.
        if HFDict.get(e) == None:
            HFDict[e] = widgets.FloatText(description='Split'+str(e+1)+' HF constant', value=1, style={'description_width': 'initial'})
        if NucleiDict.get(e) == None:   
            NucleiDict[e] = widgets.IntText(description='Split'+str(e+1)+' no. nuclei', value=1, style={'description_width': 'initial'})
        if SpinDict.get(e) == None:   
            SpinDict[e] = widgets.FloatText(description='Split'+str(e+1)+' spin', value=0.5, step=0.5, style={'description_width': 'initial'})
        
#         display(HFDict[e], NucleiDict[e], SpinDict[e])
    
    # Put all split sliders in an accordion widget that only shows the active splits.
    HBoxDict = {}
    for i in range(ActiveSplits):
        HBoxDict[i] = widgets.HBox([HFDict[i], NucleiDict[i], SpinDict[i]])
        
    SplitsVBox = widgets.VBox()
    for i in range(ActiveSplits):
         SplitsVBox.children = list(SplitsVBox.children) + [HBoxDict[i]]

    SplitsAccordion = widgets.Accordion(children=[SplitsVBox], selected_index=0)
    display(SplitsAccordion)
    
    # Add all peaks according to calculated splitting pattern.
    for e in splits: 
        peak = stats.norm.pdf(x, e[0], 0.1)
        peak *= e[1]
        y += peak
    
    if firstDeriv==True:
        y = np.gradient(y)
    
#     global fig
#     plt.ioff()
    fig = plt.figure()
    fig.clear()
    plt.plot(x, y)
    display(fig.canvas)

    
# splits = splitPeaks(1, 1, 1/2)
# splits = splitPeaks(1, 1, 1/2)
# splits = splitPeaks(1, 1, 1/2)
# splits = splitPeaks(1, 5, 5/2)




ResetButton = widgets.Button(description='Reset pattern')
SetButton = widgets.Button(description='Commit splits')
# display(ResetButton,SetButton)
ResetButton.on_click(reset)
SetButton.on_click(setPattern)

Interact = widgets.interactive(plot, 
                    resolution=widgets.BoundedIntText(value=1000, min=1, max=20000, description='Resolution', continuous_update=False), 
                    xMin = widgets.FloatSlider(value=B-10, min=-50, max=50, step=0.1, description='X min', continuous_update=False), 
                    xMax = widgets.FloatSlider(value=B+10, min=-50, max=50, step=0.1, description='X max', continuous_update=False),
                    firstDeriv = widgets.Checkbox(value=True, description='Show as 1st derivative', indent=True), 
                    NumberOfSplits = widgets.IntText(value='1', description='Number of splits', 
                                                     style={'description_width': 'initial'})
                   )
output = Interact.children[-1]
output.layout.height = '1000px'
# Interact
# display(Interact.children[0])

# display(ResetButton, SetButton, Interact)
HBox1 = widgets.HBox([Interact.children[0], Interact.children[1], Interact.children[2], Interact.children[3]])
HBox2 = widgets.HBox([Interact.children[4], ResetButton, SetButton])
VBox1 = widgets.VBox([HBox1, HBox2, Interact.children[5]])
display(VBox1)

